Notebook for Preprocessing & Saving Preprocessed Data

Loads from the RAW layer:
- stock_prices.csv 
- us_income_statements.csv 
- Merges them using SimFinId as the key.
- Saves the merged output as merged_stock_income.csv. in the PREPROCESSED Folder

This notebook logically follows the merge step and performs data cleaning & preprocessing on merged_stock_income.csv

✔ What it does:
- Loads from the RAW layer stock_prices.csv and us_income_statements.csv, and merges them using SimFinId as the key.

Then, it cleans and prepares the data:
- Drops the "Dividend" column (mostly null).
Converts "Date" to datetime.
- Handles nulls in "Shares Outstanding" intelligently using group-wise .last() and .map().
- Drops or fills missing values using thresholds, medians (for numeric), and modes (for categorical).
- Applies linear interpolation for remaining NaNs.
- Saves the cleaned version in the PREPROC folder.

In [1]:
import pandas as pd
import os

# Load the stock prices dataset
df_prices = pd.read_csv("data\RAW\stock_prices.csv") # ADAPT TO YOUR LOCAL ENVIRONMENT


# Print column names to verify existence
df_prices.info()

<>:5: SyntaxWarning: invalid escape sequence '\R'
<>:5: SyntaxWarning: invalid escape sequence '\R'
C:\Users\SLO\AppData\Local\Temp\ipykernel_26348\1986362483.py:5: SyntaxWarning: invalid escape sequence '\R'
  df_prices = pd.read_csv("data\RAW\stock_prices.csv") # ADAPT TO YOUR LOCAL ENVIRONMENT


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5871346 entries, 0 to 5871345
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Ticker              object 
 1   Date                object 
 2   SimFinId            int64  
 3   Open                float64
 4   High                float64
 5   Low                 float64
 6   Close               float64
 7   Adj. Close          float64
 8   Volume              int64  
 9   Dividend            float64
 10  Shares Outstanding  float64
dtypes: float64(7), int64(2), object(2)
memory usage: 492.7+ MB


In [2]:
# Load the income statements dataset
df_income = pd.read_csv("data/RAW/us_income_statements.csv")

df_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17555 entries, 0 to 17554
Data columns (total 26 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   SimFinId                                  17555 non-null  int64  
 1   Currency                                  17555 non-null  object 
 2   Fiscal Year                               17555 non-null  int64  
 3   Fiscal Period                             17555 non-null  object 
 4   Publish Date                              17555 non-null  object 
 5   Restated Date                             17555 non-null  object 
 6   Shares (Basic)                            17403 non-null  float64
 7   Shares (Diluted)                          17276 non-null  float64
 8   Revenue                                   15745 non-null  float64
 9   Cost of Revenue                           13770 non-null  float64
 10  Gross Profit                      

In [3]:
# Merge df_prices and df_income on the 'SimFinId' column
df_merged = pd.merge(df_prices, df_income, on="SimFinId", how="inner")

# Display info to verify merge success
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18982067 entries, 0 to 18982066
Data columns (total 36 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Dividend                                  float64
 10  Shares Outstanding                        float64
 11  Currency                                  object 
 12  Fiscal Year                               int64  
 13  Fiscal Period                             object 
 14  

In [4]:
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Dividend,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
0,A,2019-04-26,45846,76.98,77.46,76.3,77.42,74.36,1608922,NaN,...,-22000000.0,-38000000.0,9.190000e+08,NaN,9.190000e+08,152000000.0,1071000000,NaN,1071000000,1071000000
1,A,2019-04-26,45846,76.98,77.46,76.3,77.42,74.36,1608922,NaN,...,-4000000.0,-70000000.0,8.420000e+08,NaN,8.420000e+08,-123000000.0,719000000,NaN,719000000,719000000
2,A,2019-04-26,45846,76.98,77.46,76.3,77.42,74.36,1608922,NaN,...,13000000.0,-79000000.0,1.360000e+09,NaN,1.360000e+09,-150000000.0,1210000000,NaN,1210000000,1210000000
3,A,2019-04-26,45846,76.98,77.46,76.3,77.42,74.36,1608922,NaN,...,-114000000.0,-75000000.0,1.504000e+09,NaN,1.504000e+09,-250000000.0,1254000000,NaN,1254000000,1254000000
4,A,2019-04-26,45846,76.98,77.46,76.3,77.42,74.36,1608922,NaN,...,-11000000.0,-44000000.0,1.339000e+09,NaN,1.339000e+09,-99000000.0,1240000000,NaN,1240000000,1240000000


In [5]:
df_merged.columns

Index(['Ticker', 'Date', 'SimFinId', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Dividend', 'Shares Outstanding', 'Currency',
       'Fiscal Year', 'Fiscal Period', 'Publish Date', 'Restated Date',
       'Shares (Basic)', 'Shares (Diluted)', 'Revenue', 'Cost of Revenue',
       'Gross Profit', 'Operating Expenses',
       'Selling, General & Administrative', 'Research & Development',
       'Depreciation & Amortization', 'Operating Income (Loss)',
       'Non-Operating Income (Loss)', 'Interest Expense, Net',
       'Pretax Income (Loss), Adj.', 'Abnormal Gains (Losses)',
       'Pretax Income (Loss)', 'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations',
       'Net Extraordinary Gains (Losses)', 'Net Income',
       'Net Income (Common)'],
      dtype='object')

In [6]:
df_merged.describe()

,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding,Fiscal Year,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
count,1.898207e+07,1.898207e+07,1.898207e+07,1.898207e+07,1.898207e+07,1.898207e+07,1.898207e+07,107231.000000,1.830094e+07,1.898207e+07,...,1.876180e+07,1.666630e+07,1.897919e+07,1.224753e+07,1.897959e+07,1.551110e+07,1.898207e+07,1.878177e+06,1.898207e+07,1.898207e+07
mean,4.216153e+06,7.070503e+02,7.307088e+02,6.763625e+02,7.001945e+02,6.982021e+02,1.977241e+06,0.450538,2.803452e+08,2.020937e+03,...,-1.386581e+08,-2.222333e+08,5.377741e+08,-2.662387e+07,4.393831e+08,-1.208628e+08,4.246910e+08,1.133213e+08,4.228807e+08,4.193831e+08
std,5.130520e+06,4.877968e+04,5.052990e+04,4.617812e+04,4.807907e+04,4.807909e+04,3.045192e+07,1.206751,4.883713e+09,1.380871e+00,...,9.961973e+09,1.261530e+10,2.557183e+10,4.459645e+09,2.739331e+10,8.094050e+09,1.981378e+10,9.189932e+08,1.989266e+10,1.989222e+10
min,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,2.019000e+03,...,-1.007982e+12,-1.381290e+12,-1.574245e+12,-3.176900e+10,-1.574245e+12,-5.054560e+11,-1.394512e+12,-4.306000e+09,-1.409383e+12,-1.409383e+12
25%,4.213060e+05,8.990000e+00,9.240000e+00,8.710000e+00,8.970000e+00,8.500000e+00,7.537200e+04,0.130000,2.175188e+07,2.020000e+03,...,-5.200900e+07,-7.388700e+07,-2.795000e+07,-4.299600e+07,-3.865708e+07,-5.900000e+07,-3.853900e+07,-8.968000e+06,-3.812100e+07,-3.943200e+07
50%,9.952480e+05,2.436000e+01,2.491000e+01,2.380000e+01,2.434000e+01,2.304000e+01,3.444110e+05,0.260000,5.351291e+07,2.021000e+03,...,-4.800000e+06,-1.080000e+07,1.178300e+07,-4.660622e+06,6.196011e+06,-5.598000e+06,5.000000e+06,2.000000e+03,5.000000e+06,4.155000e+06
75%,9.075986e+06,6.380000e+01,6.493000e+01,6.260000e+01,6.375000e+01,6.066000e+01,1.149745e+06,0.500000,1.414653e+08,2.022000e+03,...,2.284790e+05,-2.390000e+05,2.401190e+08,6.900000e+05,2.146000e+08,-2.300000e+04,1.774080e+08,2.500000e+07,1.735460e+08,1.713630e+08
max,1.858941e+07,1.440000e+07,1.593600e+07,1.376000e+07,1.440000e+07,1.440000e+07,1.848998e+10,68.060000,4.782666e+11,2.023000e+03,...,4.258460e+11,2.017040e+11,2.710835e+12,2.809990e+11,2.710835e+12,7.366010e+11,2.205379e+12,2.182700e+10,2.201565e+12,2.201565e+12


In [7]:
df_merged.isna().mean() # 99% of the dividend column is null for all tickers 

Ticker                                      0.000000
Date                                        0.000000
SimFinId                                    0.000000
Open                                        0.000000
High                                        0.000000
Low                                         0.000000
Close                                       0.000000
Adj. Close                                  0.000000
Volume                                      0.000000
Dividend                                    0.994351
Shares Outstanding                          0.035883
Currency                                    0.000000
Fiscal Year                                 0.000000
Fiscal Period                               0.000000
Publish Date                                0.000000
Restated Date                               0.000000
Shares (Basic)                              0.005949
Shares (Diluted)                            0.011765
Revenue                                     0.

# Data Cleaning #

✅ Converted 'Date' into a Datetime Object

✅ Remove duplicate rows if needed

✅ Handled missing values


✅ Drop dividend column (with 99% of nulls)

✅ Shares outstanding -  Replace Nulls with the Latest Value Reported per Ticker 

✅ Shares outstanding -  Replace remaining nulls (for those tickers who have never reported a shares outstanding value) with 0 to preserve data integrity 

✅ Drop Columns with Too Many Missing Values (>50%)

✅ Fill Numerical Columns with Mean/Median imputation (e.g. 'Revenue', 'Cost of Revenue', 'Gross Profit', 'Operating Expenses')

✅ Fill Categorical Columns with Mode ('Currency', 'Ticker', 'Fiscal Period')

✅ Drop Rows with Too Many Nulls
If some rows still have excessive missing values, drop them.

✅ Interpolation for Time-Series Data

❌ Outliers - not needed for cleaning. 

Changing Date into a Datetime Object

In [8]:
df_merged['Date'] = pd.to_datetime(df_merged['Date']) # converting the "Date" column to a Datetime Object 

Checking for Duplicates


In [9]:
df_merged.duplicated().sum()  # Count duplicates

np.int64(0)

Dropping the "Divident" column with 99% of nulls 

In [10]:
df_merged = df_merged.drop(columns=["Dividend"])

Handling the null values in "Shares Outstanding" 

In [11]:
# Sort the DataFrame by Ticker and Date in ascending order
df_merged = df_merged.sort_values(by=["Ticker", "Date"])

# Get the latest (most recent) non-null "Shares Outstanding" per Ticker
latest_shares_outstanding = df_merged.groupby("Ticker")["Shares Outstanding"].last()

# Fill missing values using the latest reported value per Ticker
df_merged["Shares Outstanding"] = df_merged["Shares Outstanding"].fillna(df_merged["Ticker"].map(latest_shares_outstanding))

In [12]:
df_merged.isna().mean()

Ticker                                      0.000000
Date                                        0.000000
SimFinId                                    0.000000
Open                                        0.000000
High                                        0.000000
Low                                         0.000000
Close                                       0.000000
Adj. Close                                  0.000000
Volume                                      0.000000
Shares Outstanding                          0.001910
Currency                                    0.000000
Fiscal Year                                 0.000000
Fiscal Period                               0.000000
Publish Date                                0.000000
Restated Date                               0.000000
Shares (Basic)                              0.005949
Shares (Diluted)                            0.011765
Revenue                                     0.084039
Cost of Revenue                             0.

The reason some “Shares Outstanding” values are still null, even after replacing them with the latest available value, is that .last() in groupby("Ticker") selects only the most recent non-null value per Ticker. However, if a Ticker never had a reported non-null value, .last() returns NaN instead of a valid number. As a result, when we try to map and fill missing values, there is no valid value available for replacement, leaving some entries still null.

In [13]:
null_so =  df_merged[df_merged["Shares Outstanding"].isna()]

null_so

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
397440,ADRA,2019-04-26,11817231,32.02,32.04,31.98,31.98,31.25,947,NaN,...,4307581.0,10281.0,3330750.0,-86544.0,3244206.0,NaN,3244206,NaN,3244206,3244206
397441,ADRA,2019-04-26,11817231,32.02,32.04,31.98,31.98,31.25,947,NaN,...,-4056000.0,-4056000.0,47697000.0,-9656000.0,38041000.0,-9423000.0,28618000,NaN,28618000,28618000
397442,ADRA,2019-04-26,11817231,32.02,32.04,31.98,31.98,31.25,947,NaN,...,-11715000.0,-11715000.0,-36308000.0,-8154000.0,-44462000.0,9058000.0,-35404000,NaN,-35404000,-35404000
397443,ADRA,2019-04-29,11817231,32.01,32.15,32.00,32.15,31.41,4147,NaN,...,4307581.0,10281.0,3330750.0,-86544.0,3244206.0,NaN,3244206,NaN,3244206,3244206
397444,ADRA,2019-04-29,11817231,32.01,32.15,32.00,32.15,31.41,4147,NaN,...,-4056000.0,-4056000.0,47697000.0,-9656000.0,38041000.0,-9423000.0,28618000,NaN,28618000,28618000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18099245,VTOL,2024-03-27,10656416,26.21,26.78,26.21,26.78,26.78,79213,NaN,...,-4593000.0,-41360000.0,28154000.0,-32573000.0,-4419000.0,-11294000.0,-15713000,NaN,-15791000,-15791000
18099246,VTOL,2024-03-27,10656416,26.21,26.78,26.21,26.78,26.78,79213,NaN,...,-35574000.0,-32771000.0,19101000.0,-1089000.0,18012000.0,-24932000.0,-6920000,NaN,-6780000,-6780000
18099247,VTOL,2024-03-28,10656416,26.96,27.38,26.91,27.20,27.20,119852,NaN,...,-22045000.0,-49966000.0,42707000.0,-99347000.0,-56640000.0,355000.0,-56285000,NaN,-56094000,-56094000
18099248,VTOL,2024-03-28,10656416,26.96,27.38,26.91,27.20,27.20,119852,NaN,...,-4593000.0,-41360000.0,28154000.0,-32573000.0,-4419000.0,-11294000.0,-15713000,NaN,-15791000,-15791000


Replacing the Shares Outstanding Values that have never had a reported non-null vallue with 0. 

In [14]:
df_merged["Shares Outstanding"] = df_merged["Shares Outstanding"].fillna(0) # replacing the remaining "Shares Outstanding" with 0

In [15]:
null_means = df_merged.isna().mean()
for column, mean in null_means.items():
    if mean > 0:
        print(f"{column}: {mean:.6f}")

Shares (Basic): 0.005949
Shares (Diluted): 0.011765
Revenue: 0.084039
Cost of Revenue: 0.196228
Gross Profit: 0.196173
Operating Expenses: 0.001840
Selling, General & Administrative: 0.053792
Research & Development: 0.539041
Depreciation & Amortization: 0.585322
Operating Income (Loss): 0.000152
Non-Operating Income (Loss): 0.011604
Interest Expense, Net: 0.121998
Pretax Income (Loss), Adj.: 0.000152
Abnormal Gains (Losses): 0.354784
Pretax Income (Loss): 0.000131
Income Tax (Expense) Benefit, Net: 0.182855
Net Extraordinary Gains (Losses): 0.901055


Drop Columns with Too Many Missing Values (>50%)
If a column has too many missing values and isn't critical, it's best to drop it.

columns dropped: Research & Development, Depreciation & Amortization

In [16]:
threshold = 0.5  # 50% missing values threshold
df_merged = df_merged.dropna(axis=1, thresh=len(df_merged) * (1 - threshold))

Fill Numerical Columns with Mean/Median
For numerical data like Revenue, Gross Profit, etc., use median imputation.
Used median imputation instead of mean because median is more robust to outliers

In [17]:
num_cols = ['Revenue', 'Cost of Revenue', 'Gross Profit', 'Operating Expenses']
for col in num_cols:
    df_merged[col].fillna(df_merged[col].median(), inplace=True)

C:\Users\SLO\AppData\Local\Temp\ipykernel_26348\2202505327.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged[col].fillna(df_merged[col].median(), inplace=True)
C:\Users\SLO\AppData\Local\Temp\ipykernel_26348\2202505327.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


Fill Categorical Columns with Mode
For categorical columns like Currency, Ticker, or Fiscal Period.

- Categorical columns like Currency, Ticker, and Fiscal Period contain discrete values.
- The mode is the most frequently occurring category, making it a logical replacement for missing values.
- If we later use one-hot encoding or label encoding, missing values could create problems.
- Filling with the mode ensures that all rows have valid categorical values.

In [18]:
cat_cols = ['Currency', 'Ticker', 'Fiscal Period']
for col in cat_cols:
    df_merged[col].fillna(df_merged[col].mode()[0], inplace=True)


C:\Users\SLO\AppData\Local\Temp\ipykernel_26348\1632782131.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged[col].fillna(df_merged[col].mode()[0], inplace=True)


Drop Rows with Too Many Nulls
If some rows still have excessive missing values, drop them.

In [19]:
df_merged.dropna(thresh=len(df_merged.columns) * 0.7, inplace=True)  # Drops rows with >30% missing

Interpolation for Time-Series Data

Uses linear interpolation to estimate missing values between known values.
Works by computing a straight-line equation between two known points and filling missing values accordingly.
- If your dataset follows a time pattern (e.g., financial data), use interpolation.
- For time-series financial data (e.g., stock prices, revenue), missing values often occur due to holidays, reporting delays, or system gaps.
- Interpolation helps smooth data trends rather than using arbitrary imputation like mean/median.
- It ensures that missing values follow the trend instead of creating unnatural spikes.

In [20]:
df_merged = df_merged.sort_values('Date')  # Ensure sorting before interpolation
df_merged.interpolate(method='linear', inplace=True)

C:\Users\SLO\AppData\Local\Temp\ipykernel_26348\2709780628.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_merged.interpolate(method='linear', inplace=True)


In [21]:
df_merged.isna().mean()

Ticker                                      0.000000e+00
Date                                        0.000000e+00
SimFinId                                    0.000000e+00
Open                                        0.000000e+00
High                                        0.000000e+00
Low                                         0.000000e+00
Close                                       0.000000e+00
Adj. Close                                  0.000000e+00
Volume                                      0.000000e+00
Shares Outstanding                          0.000000e+00
Currency                                    0.000000e+00
Fiscal Year                                 0.000000e+00
Fiscal Period                               0.000000e+00
Publish Date                                0.000000e+00
Restated Date                               0.000000e+00
Shares (Basic)                              0.000000e+00
Shares (Diluted)                            0.000000e+00
Revenue                        

In [22]:
os.makedirs("data/PREPROCESSING", exist_ok=True)

In [23]:
# Save merged dataset
df_merged.to_csv("data/PREPROCESSING/merged_stock_income.csv", index=False) 